In [1]:
import pandas as pd
import re

df = pd.read_excel('gurugram_property.xlsx')
print("Initial Data:")
print(df.info())
print(df.head())

Initial Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sector     1445 non-null   int64 
 1   size       1445 non-null   int64 
 2   Title      1440 non-null   object
 3   Price      1445 non-null   object
 4   Area_SqFt  1445 non-null   object
 5   link       1445 non-null   object
dtypes: int64(2), object(4)
memory usage: 67.9+ KB
None
   sector  size                       Title       Price   Area_SqFt  \
0     102     1                 ROF AALAYAS     ₹38 Lac    403 sqft   
1     102     1                 ROF AALAYAS  ₹37.25 Lac    430 sqft   
2     102     1                 ROF AALAYAS   ₹37.5 Lac    410 sqft   
3     102     1  SIGNATURE GLOBAL CITY 7...    ₹1.72 Cr  1,325 sqft   
4     102     1      CONSCIENT HERITAGE MAX     ₹20 Lac    200 sqft   

                                                link  
0  https://www.99acres.com/1-b

In [ ]:
def clean_price(price_str):
    # Ensure input is a string and make it lowercase
    price_str = str(price_str).lower()
    
    try:
        # Find the first number (integer or decimal) in the string
        num_str = re.findall(r'[\d\.]+', price_str)[0]
        num = float(num_str)
        
        # Check for 'cr' (crore) and multiply accordingly
        if 'cr' in price_str:
            return num * 1_00_00_000
        # Check for 'lac' (lakh) and multiply accordingly
        elif 'lac' in price_str:
            return num * 1_00_000
        # If no 'cr' or 'lac', return the number as is (assuming it's already in Rupees)
        else:
            # Also handle cases with commas, like '12,50,00,000'
            just_digits = re.sub(r'[^\d]', '', price_str)
            return float(just_digits)
            
    except (IndexError, ValueError):
        # If no number is found or conversion fails (e.g., "Price on Request")
        return pd.NA 


In [ ]:
def clean_area(area_str):
    # Ensure input is a string
    area_str = str(area_str)
    
    try:
        # First, remove commas from the string to handle values like "1,325"
        area_str_no_commas = area_str.replace(',', '')
        
        # Find the first number (integer or decimal)
        num_str = re.findall(r'[\d\.]+', area_str_no_commas)[0]
        return float(num_str)
    except (IndexError, ValueError):
        # If no number is found (e.g., "N/A")
        return pd.NA

In [5]:
# Apply the cleaning functions
df['Price_Rupees'] = df['Price'].apply(clean_price)
df['Area_SqFt'] = df['Area_SqFt'].apply(clean_area)

# # Drop rows where we couldn't clean the data properly
df.dropna(subset=['Price_Rupees', 'Area_SqFt'], inplace=True)

# # Select and rename columns for the final clean dataset
df_clean = df[['sector', 'Title','size', 'Area_SqFt', 'Price_Rupees','link']].copy()
df_clean = df_clean.dropna()

# print("\nCleaned Data:")
print(df_clean.info())
print(df_clean.head())

# Save the cleaned data to a new CSV file
df_clean.to_excel('gurugram_property_clean.xlsx', index=False)
print("\nClean data saved to gurugram_properties_clean.xlsx")

<class 'pandas.core.frame.DataFrame'>
Index: 1440 entries, 0 to 1444
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sector        1440 non-null   int64  
 1   Title         1440 non-null   object 
 2   size          1440 non-null   int64  
 3   Area_SqFt     1440 non-null   float64
 4   Price_Rupees  1440 non-null   float64
 5   link          1440 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 78.8+ KB
None
   sector                       Title  size  Area_SqFt  Price_Rupees  \
0     102                 ROF AALAYAS     1      403.0     3800000.0   
1     102                 ROF AALAYAS     1      430.0     3725000.0   
2     102                 ROF AALAYAS     1      410.0     3750000.0   
3     102  SIGNATURE GLOBAL CITY 7...     1     1325.0    17200000.0   
4     102      CONSCIENT HERITAGE MAX     1      200.0     2000000.0   

                                                link  